In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import matrix_dev
import matplotlib.pyplot as plt
import _pickle
import os
%matplotlib inline

Get raw documents

In [ ]:
raw_docs = matrix_dev.matrix_dev('supertrain').raw_docs()
dev = matrix_dev.matrix_dev('superdev').raw_docs()

In [2]:
# #save to pickle
# path = os.path.join("../../feature_groups/lda_pickles", 'raw_docs')
# with open(path, "wb") as f:
#     _pickle.dump(raw_docs, f)
# path = os.path.join("../../feature_groups/lda_pickles", 'dev')
# with open(path, "wb") as f:
#     _pickle.dump(dev, f)
    
    
    
#load pickle
path = os.path.join("../../feature_groups/lda_pickles", 'raw_docs')
with open(path, "rb") as f:
    raw_docs = _pickle.load(f)
path = os.path.join("../../feature_groups/lda_pickles", 'dev')
with open(path, "rb") as f:
    dev = _pickle.load(f)

LDA model

In [3]:
# get tf-idf counts from each document
no_features = 1000
tf_vectorizer = TfidfVectorizer(max_features=no_features) #stop_words='english')
tf = tf_vectorizer.fit_transform(raw_docs)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
path = os.path.join("../../feature_groups/lda_pickles", 'tf_vectorizer')
with open(path, "wb") as f:
    _pickle.dump(tf, f)

In [ ]:
#sparcity check
data_dense = tf.todense()
sparcity = ((data_dense > 0).sum()/data_dense.size) * 100
print(sparcity)

In [ ]:
#Train LDA model
search_params = {'n_components':[35, 70, 140], 'learning_decay':[0.5, 0.7, 0.9]}


lda = LatentDirichletAllocation(n_components=70, max_iter=10000,learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

In [ ]:
#save to pickle
path = os.path.join("../../feature_groups/lda_pickles", file+'_lda')
with open(path, "wb") as f:
    _pickle.dump(lda, f)

In [ ]:
print(tf)

In [ ]:
#get train the perplexity
train_gamma = lda.transform(tf)
train_perplexity = lda.perplexity(tf, train_gamma)
print(train_perplexity)

In [ ]:
#log likelihood
logl = lda.score(tf)
print(logl)

In [ ]:
#look an the development set 
X_test = tf_vectorizer.fit_transform(dev)
doc_topic_dist_unnorm = np.matrix(lda.transform(X_test))

In [ ]:
#get test
dev_gamma = lda.transform(X_test)
dev_perplexity = lda.perplexity(X_test, dev_gamma)
print(dev_perplexity)

In [ ]:
doc_topic_dist = doc_topic_dist_unnorm/doc_topic_dist_unnorm.sum(axis=1)

In [ ]:
# get the top ranking topic for each document in the dev set
top_ranking = doc_topic_dist.argmax(axis=1)

In [ ]:
print(top_ranking)

In [ ]:
plt.hist(doc_topic_dist)

In [ ]:
plt.hist(top_ranking)

In [ ]:
path = os.path.join("../../feature_groups/lda_pickles", file+'_lda')
with open(path, "rb") as f:
    r = _pickle.load(f)

In [ ]:
type(r)